In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

input_dir = Path("res")
input_dir.mkdir(exist_ok=True)

csv_files = list(input_dir.glob("*.csv"))

if not csv_files:
    print("Aucun fichier CSV trouvé dans le dossier 'res'.")
else:
    for csv_path in csv_files:
        try:
            df = pd.read_csv(csv_path)

            # Vérifs basiques
            if "locality" not in df.columns or "trend_score" not in df.columns:
                print(f"⚠️ {csv_path.name} doit contenir les colonnes 'locality' et 'trend_score'. Ignoré.")
                continue

            # Forcer numérique (si jamais il y a des strings), conserver NaN pour éviter plantage
            df["trend_score"] = pd.to_numeric(df["trend_score"], errors="coerce")

            # Séparer par signe
            pos_mask = df["trend_score"] > 0
            neg_mask = df["trend_score"] < 0
            zero_mask = df["trend_score"] == 0

            # Max positif et min négatif (le plus petit, donc le plus négatif)
            max_pos = df.loc[pos_mask, "trend_score"].max() if pos_mask.any() else None
            min_neg = df.loc[neg_mask, "trend_score"].min() if neg_mask.any() else None  # ex: -18.16

            # Créer une copie pour travailler proprement
            ts = df["trend_score"].copy()

            # Positifs → [0, 10] (si au moins un positif et max_pos != 0)
            if max_pos is not None and max_pos != 0:
                ts.loc[pos_mask] = 10 * df.loc[pos_mask, "trend_score"] / max_pos

            # Négatifs → [-10, 0] (si au moins un négatif et min_neg != 0)
            if min_neg is not None and min_neg != 0:
                # min_neg est négatif, on obtient bien -10 au min, proche de 0 pour faibles valeurs
                ts.loc[neg_mask] = -10 * df.loc[neg_mask, "trend_score"] / min_neg

            # Zéros inchangés
            ts.loc[zero_mask] = 0

            # Si un côté n’existe pas (ex. que des positifs), on laisse l’autre côté tel quel.
            # Cas limites: si max_pos==0 (tous 0) ou min_neg==0 (n’arrive pas logiquement), tout reste 0.

            # Remplacer la colonne trend_score par la version normalisée
            df["trend_score"] = ts

            # Ne garder que les deux colonnes demandées et dans le bon ordre
            df_out = df[["locality", "trend_score"]]

            # Enregistrement
            output_path = csv_path.with_name(csv_path.stem + "_norma.csv")
            df_out.to_csv(output_path, index=False)
            print(f"✅ Normalisé (signe préservé) → {output_path.name}")

        except Exception as e:
            print(f"❌ Erreur avec {csv_path.name} : {e}")


✅ Fichier normalisé enregistré : restau_trend_score_norma.csv
✅ Fichier normalisé enregistré : third_sector_establishment_score_by_locality_norma.csv
✅ Fichier normalisé enregistré : third_sector_job_score_by_locality_norma.csv
✅ Fichier normalisé enregistré : demo_trend_score_norma.csv
